In [1]:
import sys; sys.path.append("../")
from utils_train import load_from_checkpoint, transfer_batch_to_device
from utils_analysis import *
from dataset_wrappper import NewsData
import torch
from loss_and_optimisation import approximate_log_q_z

Not deleting all pareto related files, if you want to recompute, run: update(recompute=True)
Making run overview, based on dir: /home/cbarkhof/code-thesis/NewsVAE/Runs
Making run overview of /home/cbarkhof/code-thesis/NewsVAE/Runs, in /home/cbarkhof/code-thesis/NewsVAE/final-analysis/Runs_run_overview.csv
pareto epoch not in full_par_dict for run: 2021-06-02-YELP | DECODER-ONLY-run-13:01:04
pareto epoch not in full_par_dict for run: 2021-06-02-PTB | DECODER-ONLY-run-13:01:37
Reading last checkpoint and extracting pareto dict and saving it to a pickle.
Reading all pareto dicts and calculating best checkpoint, saving it to a csv
error in calc_weighted_pareto_best_checkpoint list index out of range
error in calc_weighted_pareto_best_checkpoint list index out of range
error in calc_weighted_pareto_best_checkpoint list index out of range
--------------------------------------------------
Making run overview, based on dir: /home/cbarkhof/code-thesis/NewsVAE/Runs-ablation
Making run overview 

In [3]:
BATCH_SIZE = 64
IW_N_SAMPLES = 200
VAL_BATCHES = 10
DEVICE = "cuda:0"

In [ ]:
for exp_name, run_dir in RUN_DIRS.items():
    
    print("EXP:", exp_name)
    
    runs_df = read_overview_csv(exp_name=exp_name)
    
    for row_idx, row in runs_df.iterrows():
        run_name = row["run_name"]

        if check_if_running(run_name, exp_name):
            continue
        
        print("*" * 50)
        print(run_name)
        print("*" * 50)
        
        try:
            #--------------------------------------------------
            # Save the stats to
            result_file = f"{RES_FILE_DIR}/{exp_name}/{run_name}/validation_results_{VAL_BATCHES}_batches_{IW_N_SAMPLES}_samples_BS_{BATCH_SIZE}.p"
            if os.path.isfile(result_file):
                print("Did this one already, continuing.")
                continue

            #--------------------------------------------------
            # Get data that this model was trained on
            if "optimus" in row["dataset"].lower():
                dataset_name = "optimus_yelp"
            elif "yelp" in row["dataset"].lower():
                dataset_name = "yelp"
            else:
                dataset_name = "ptb_text_only"

            # Load relevant data
            data = NewsData(dataset_name=dataset_name, tokenizer_name="roberta", batch_size=BATCH_SIZE, 
                            num_workers=3, pin_memory=True, max_seq_len=64, device=DEVICE)
            val_loader = data.val_dataloader(shuffle=False, batch_size=BATCH_SIZE)

            # Determine number of batches to evaluate
            if VAL_BATCHES > len(val_loader) or VAL_BATCHES < 0:
                MAX_BATCHES = len(val_loader)
            else:
                MAX_BATCHES = VAL_BATCHES

            #--------------------------------------------------
            # Get best checkpoint and loss term manager
            path, best_epoch = get_best_checkpoint(run_name, exp_name=exp_name)
            loss_term_manager = load_from_checkpoint(path, world_master=True, ddp=False, device_name="cuda:0", 
                                                     evaluation=True, return_loss_term_manager=True)

            res = {
                "lens_x_gen":[], # batch
                "lens":[],

                "iw_ll_x_gen_p_w":[], # batch
                "iw_ll_x_gen": [],
                "iw_ll_p_w": [],
                "iw_ll": [],

                "reconstruction_loss": [], # float
                "exact_match": [],
                "cross_entropy_per_word": [],

                "log_q_z_x":[], # batch x samples
                "log_p_z":[],
                "log_p_x_z":[],

                "latent_z": [], # batch x latent_dim
                "mu":[],
                "logvar": [],

                "kl_analytical": [], # float
                "fb_kl_analytical": [],
                "elbo": [],
                "marginal KL": [],
                "dim_KL": [], 
                "TC": [],
                "MI": [],
                "mmd_loss":[],

                "std_x_std": [], # float
                "mean_x_std": [],
                "std_x_mu": [],
                "mean_x_mu": [],
                "std_z_std": [],
                "std_z_mu":[],
            }


            #--------------------------------------------------
            # Make predictions
            for batch_i, batch in enumerate(val_loader):
                print(f"Batch {batch_i+1:3d}/{MAX_BATCHES:3d}", end="\r")

                batch = transfer_batch_to_device(batch)

                with torch.no_grad():
                    out = loss_term_manager(batch["input_ids"], batch["attention_mask"], 
                                            return_exact_match=True, decoder_only=False, eval_iw_ll_x_gen=True,
                                            return_posterior_stats=True, device_name=DEVICE, iw_ll_n_samples=IW_N_SAMPLES,
                                            return_attention_to_latent=False, train=False, max_seq_len_x_gen=64)

                for k, v in out.items():
                    if k in res:
                        if torch.is_tensor(v):
                            res[k].append(v.cpu())
                        else:
                            res[k].append(v)

                if batch_i + 1 == MAX_BATCHES:
                    break

            #--------------------------------------------------
            # Gather predictions
            for k, v in res.items():
                if torch.is_tensor(v[0]):
                    if v[0].dim() > 0:
                        res[k] = torch.cat(v).cpu()
                    else:
                        res[k] = torch.stack(v).cpu()

            #--------------------------------------------------
            # Add log q z on whole validation set (only one latent sample z per data point x is needed)
            res["log_q_z"], _ = approximate_log_q_z(res["mu"], res["logvar"], res["latent_z"][:, 0, :], method="all", prod_marginals=False, reduce_mean=False)
            res["log_q_z_mean"] = res["log_q_z"].mean()

            #--------------------------------------------------
            # Dump the result in a pickle
            pickle.dump(res, open(result_file, "wb"))
        
        except Exception as e:
            print("** ERROR for :", run_name, e)

EXP: Runs
**************************************************
2021-05-31-YELP | MDR-0.5 | matrix-memory | DROP 40-run-06:30:27
**************************************************
Did this one already, continuing.
**************************************************
2021-05-24-PTB | CYC-FB-0.5 | matrix-memory-run-19:12:45
**************************************************
Did this one already, continuing.
**************************************************
2021-05-26-YELP | VAE | matrix-run-20:28:49
**************************************************
Did this one already, continuing.
**************************************************
2021-05-30-YELP | CYC-FB-0.5 | matrix-memory-run-04:07:55
**************************************************
Did this one already, continuing.
**************************************************
2021-05-31-YELP | MDR-0.5 | memory-embeddings | DROP 40-run-15:27:52
**************************************************
Did this one already, continuing.
*****************

**************************************************
2021-05-24-PTB | VAE | matrix | DROP 40-run-12:08:05
**************************************************
Did this one already, continuing.
**************************************************
2021-05-31-PTB | CYC-FB-0.5 | matrix-run-17:33:15
**************************************************
Did this one already, continuing.
**************************************************
2021-05-24-PTB | VAE | matrix-memory | DROP 40-run-15:34:33
**************************************************
Did this one already, continuing.
**************************************************
2021-05-26-PTB | MDR-0.5 | matrix-memory | DROP 40-run-09:00:48
**************************************************
Did this one already, continuing.
**************************************************
2021-05-29-YELP | CYC-FB-0.5 | matrix | DROP 40-run-08:07:49
**************************************************
Did this one already, continuing.
********************************

Some weights of the model checkpoint at roberta-base were not used when initializing VaeDecoderRobertaForCausalLM: ['lm_head.bias']
- This IS expected if you are initializing VaeDecoderRobertaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VaeDecoderRobertaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VaeDecoderRobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.0.attention.self.query_module.alpha', 'roberta.encoder.layer.0.attention.self.query_module.layer.weight', 'roberta.encoder.layer.0.attention.self.query_module.layer.bias', 'roberta.encoder.layer.0.attention.self.key_mo

Some weights of VAE_Encoder_RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Done loading model...
********************************************************************************
Free bits pd 0.0
********************************************************************************
Checkpoint global_step: 8416, epoch: 7
Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Setting to eval mode.


...

Some weights of the model checkpoint at roberta-base were not used when initializing VaeDecoderRobertaForCausalLM: ['lm_head.bias']
- This IS expected if you are initializing VaeDecoderRobertaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VaeDecoderRobertaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VaeDecoderRobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.0.attention.self.query_module.layer.weight', 'roberta.encoder.layer.0.attention.self.query_module.layer.bias', 'roberta.encoder.layer.0.attention.self.key_module.layer.weight', 'roberta.encoder.layer.0.attention.self.k

Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Done loading model...
********************************************************************************
Free bits pd 0.0
********************************************************************************
Checkpoint global_step: 5922, epoch: 17
Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Setting to eval mode.
***********

Some weights of the model checkpoint at roberta-base were not used when initializing VaeDecoderRobertaForCausalLM: ['lm_head.bias']
- This IS expected if you are initializing VaeDecoderRobertaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VaeDecoderRobertaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VaeDecoderRobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.0.attention.self.query_module.alpha', 'roberta.encoder.layer.0.attention.self.query_module.layer.weight', 'roberta.encoder.layer.0.attention.self.query_module.layer.bias', 'roberta.encoder.layer.0.attention.self.key_mo

Some weights of VAE_Encoder_RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Done loading model...
********************************************************************************
Free bits pd 0.0
********************************************************************************
Checkpoint global_step: 100010, epoch: 9
Tying encoder decoder RoBERTa checkpoint weights!
<class 'modules.decoder_roberta.VaeDecoderRobertaModel'> and <class 'modules.encoder_roberta.VAE_Encoder_RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Tying embedding spaces!
Setting to eval mode.
